In [41]:
!pip install owlready2

In [42]:
from owlready2 import *

In [43]:
onto = get_ontology("scenarios_kgs_merged.owl").load()

In [44]:
hi = onto.get_namespace("https://w3id.org/hi-ontology#")

In [45]:
with onto:

    class HITeam(Thing):
        pass

    class Goal(Thing):
        pass

    class Task(Thing):
        pass

    class Agent(Thing):
        pass

    class Capability(Thing):
        pass

    class TaskExecution(Thing):
        pass

    class Context(Thing):
        pass

    # Properties needed for all rules
    class hasGoal(ObjectProperty):
        domain = [HITeam]
        range  = [Goal]

    class requiresTask(ObjectProperty):
        domain = [Goal]
        range  = [Task]

    class hasMember(ObjectProperty):
        domain = [HITeam]
        range  = [Agent]

    class hasCapability(ObjectProperty):
        domain = [Agent]
        range  = [Capability]

    class allowsTask(ObjectProperty):
        domain = [Capability]
        range  = [Task]

    class isEligibleForTask(ObjectProperty):
        domain = [Agent]
        range  = [Task]

    class isAssignedToTask(ObjectProperty):
        domain = [Agent]
        range  = [Task]

    class realizedBy(ObjectProperty):
        domain = [Task]
        range  = [TaskExecution]

    class performedBy(ObjectProperty):
        domain = [TaskExecution]
        range  = [Agent]

    class operatesInContext(ObjectProperty):
        domain = [HITeam]
        range  = [Context]

    class hasPhenomenonConcept(ObjectProperty):
        domain = [Context, Agent]
        range  = [Thing]

    class hasConstraintConcept(ObjectProperty):
        domain = [Context, Agent]
        range  = [Thing]


## RULE 1


In [46]:
with onto:
    rule1 = Imp()
    rule1.set_as_rule("""
        HITeam(?team),
        hasGoal(?team, ?goal),
        requiresTask(?goal, ?task),
        hasMember(?team, ?agent),
        hasCapability(?agent, ?cap),
        allowsTask(?cap, ?task)
        ->
        isEligibleForTask(?agent, ?task)
    """)

## RULE 2

In [47]:
with onto:
    rule2 = Imp()
    rule2.set_as_rule("""
        performedBy(?task, ?agent)
        ->
        isAssignedToTask(?agent, ?task)
    """)

In [48]:
with onto:
    rule2 = Imp()
    rule2.set_as_rule("""
        isAssignedToTask(?agent, ?task),
        realizedBy(?task, ?exec)
        ->
        performedBy(?exec, ?agent)
    """)


## RULE 3a


In [49]:
with onto:
    rule3a = Imp()
    rule3a.set_as_rule("""
        operatesInContext(?team, ?context),
        hasMember(?team, ?agent),
        hasPhenomenonConcept(?context, ?phen)
        ->
        hasPhenomenonConcept(?agent, ?phen)
    """)


## RULE 3b


In [50]:
with onto:
    rule3b = Imp()
    rule3b.set_as_rule("""
        operatesInContext(?team, ?context),
        hasMember(?team, ?agent),
        hasConstraintConcept(?context, ?cons)
        ->
        hasConstraintConcept(?agent, ?cons)
    """)


## RULE 4

In [51]:
with onto:
    rule4 = Imp()
    rule4.set_as_rule("""
        isAssignedToTask(?agent, ?task)
        ->
        isEligibleForTask(?agent, ?task)
    """)

## RUN REASONER (Hermit)

In [52]:
sync_reasoner_pellet([onto], infer_property_values=True)

* Owlready2 * Running Pellet...
    java -Xmx2000M -cp /usr/local/lib/python3.12/dist-packages/owlready2/pellet/pellet-2.3.1.jar:/usr/local/lib/python3.12/dist-packages/owlready2/pellet/owlapi-distribution-3.4.3-bin.jar:/usr/local/lib/python3.12/dist-packages/owlready2/pellet/log4j-api-2.19.0.jar:/usr/local/lib/python3.12/dist-packages/owlready2/pellet/jena-core-2.10.0.jar:/usr/local/lib/python3.12/dist-packages/owlready2/pellet/jgrapht-jdk1.5.jar:/usr/local/lib/python3.12/dist-packages/owlready2/pellet/jcl-over-slf4j-1.6.4.jar:/usr/local/lib/python3.12/dist-packages/owlready2/pellet/jena-tdb-0.10.0.jar:/usr/local/lib/python3.12/dist-packages/owlready2/pellet/jena-iri-0.9.5.jar:/usr/local/lib/python3.12/dist-packages/owlready2/pellet/jena-arq-2.10.0.jar:/usr/local/lib/python3.12/dist-packages/owlready2/pellet/xml-apis-1.4.01.jar:/usr/local/lib/python3.12/dist-packages/owlready2/pellet/httpclient-4.2.3.jar:/usr/local/lib/python3.12/dist-packages/owlready2/pellet/antlr-3.2.jar:/usr/local

In [53]:
eligible_property = onto.isEligibleForTask

print("\n=== Inferred Eligible Agents ===\n")
for agent, task in eligible_property.get_relations():
    print(f"{agent.name}  -->  {task.name}")


=== Inferred Eligible Agents ===

HumanLearner  -->  TaskReconstructTrajectory
AITutor  -->  TaskExplainRecommendations
AITutor  -->  TaskBayesianInference
AITutor  -->  TaskRecommendResources
AITutor  -->  TaskKGQuery
AITutor  -->  TaskSemanticReasoning
RobotManipulator  -->  TaskOpenCabinet
RobotManipulator  -->  TaskRetrieveTool
HumanUser  -->  TaskDataCollection
HumanVisitor  -->  TaskInterpretMultimodalInput
AIAssistant  -->  TaskExplanation
AIAssistant  -->  TaskDataCollection
AIAssistant  -->  TaskConflictResolution
AIDeliberationAssistant  -->  TaskDecideIntervention
AIDeliberationAssistant  -->  TaskAssessCommunicationAccuracy
AIDeliberationAssistant  -->  TaskExplainDecision
HumanClinician  -->  TaskHypothesisComparison
HumanClinician  -->  TaskExplainReasoning
HumanTechnician  -->  TaskUnlockCabinet
HumanJuror1  -->  TaskProvideStatement
HumanJuror2  -->  TaskProvideStatement
AIVirtualGuide  -->  TaskSemanticRetrieval
AIVirtualGuide  -->  TaskRecommendArtifacts
AIVirtualGui

In [54]:
for agent, task in onto.isAssignedToTask.get_relations():
    print(agent, task)

4.HumanLearner 4.ExecProvideLearningHistory
4.AITutor 4.ExecSemanticReasoning
4.AITutor 4.ExecExplainRecommendations
4.AITutor 4.ExecRecommendResources
4.AITutor 4.ExecBayesianModeling
4.AITutor 4.ExecReconstructTrajectory
2.RobotManipulator 2.ExecOpenCabinet
2.RobotManipulator 2.ExecRLExplain
2.RobotManipulator 2.ExecConfinedReach
2.RobotManipulator 2.ExecHeavyLift
1.HumanUser 1.ExecutionDataCollection
7.HumanVisitor 7.ExecProvideProfile
1.AIAssistant 1.ExecutionExplanation
1.AIAssistant 1.ExecutionConflictResolution
3.AIDeliberationAssistant 3.ExecExplainDecision
3.AIDeliberationAssistant 3.ExecDecideIntervention
3.AIDeliberationAssistant 3.ExecAssessAccuracy
5.HumanClinician 5.ExecExplainHuman
5.HumanClinician 5.ExecHypothesisComparison
2.HumanTechnician 2.ExecUnlockCabinet
3.HumanJuror1 3.ExecProvideStatement1
3.HumanJuror2 3.ExecProvideStatement2
7.AIVirtualGuide 7.ExecRecommendArtifacts
7.AIVirtualGuide 7.ExecComputeSimilarity
7.AIVirtualGuide 7.ExecSemanticRetrieval
7.AIVirtualG

In [55]:
print("\n=== Agents influenced by PHENOMENA (Rule 3a) ===\n")

for agent in onto.Agent.instances():
    if hasattr(agent, "hasPhenomenonConcept") and agent.hasPhenomenonConcept:
        print(f"{agent.name}:")
        for phen in agent.hasPhenomenonConcept:
            print(f"   - {phen.name}")


=== Agents influenced by PHENOMENA (Rule 3a) ===

AIAssistant:
   - SensorNoisePhenomenon
   - TrustShiftPhenomenon
   - DataMismatchPhenomenon
ContextHealthMonitoring:
   - DataMismatchPhenomenon
   - SensorNoisePhenomenon
   - TrustShiftPhenomenon
HumanUser:
   - SensorNoisePhenomenon
   - TrustShiftPhenomenon
   - DataMismatchPhenomenon
ContextWorkspaceManipulation:
   - RiskExposurePhenomenon
   - SkillsComplementarityPhenomenon
RobotManipulator:
   - RiskExposurePhenomenon
   - SkillsComplementarityPhenomenon
HumanTechnician:
   - RiskExposurePhenomenon
   - SkillsComplementarityPhenomenon
AIDeliberationAssistant:
   - GroupThinkingPhenomenon
   - TrustShiftPhenomenon
ContextJuryDeliberation:
   - GroupThinkingPhenomenon
   - TrustShiftPhenomenon
HumanJuror1:
   - GroupThinkingPhenomenon
   - TrustShiftPhenomenon
HumanJuror2:
   - GroupThinkingPhenomenon
   - TrustShiftPhenomenon
AITutor:
   - KnowledgeMismatchPhenomenon
   - TrustShiftPhenomenon
ContextOnlineLearning:
   - Knowl

In [56]:
print("\n=== Agents influenced by CONSTRAINTS (Rule 3b) ===\n")

for agent in onto.Agent.instances():
    if hasattr(agent, "hasConstraintConcept") and agent.hasConstraintConcept:
        print(f"{agent.name}:")
        for cons in agent.hasConstraintConcept:
            print(f"   - {cons.name}")



=== Agents influenced by CONSTRAINTS (Rule 3b) ===

AIAssistant:
   - PrivacyConstraint
   - AutonomyConstraint
   - TransparencyConstraint
ContextHealthMonitoring:
   - AutonomyConstraint
   - PrivacyConstraint
   - TransparencyConstraint
HumanUser:
   - PrivacyConstraint
   - AutonomyConstraint
   - TransparencyConstraint
ContextWorkspaceManipulation:
   - SafetyConstraint
   - TransparencyConstraint
RobotManipulator:
   - SafetyConstraint
   - TransparencyConstraint
HumanTechnician:
   - SafetyConstraint
   - TransparencyConstraint
AIDeliberationAssistant:
   - UndueInfluenceConstraint
   - FairnessConstraint
   - TransparencyConstraint
ContextJuryDeliberation:
   - FairnessConstraint
   - TransparencyConstraint
   - UndueInfluenceConstraint
HumanJuror1:
   - UndueInfluenceConstraint
   - FairnessConstraint
   - TransparencyConstraint
HumanJuror2:
   - UndueInfluenceConstraint
   - FairnessConstraint
   - TransparencyConstraint
AITutor:
   - PrivacyConstraint
   - AutonomyConstrain

In [72]:
# Select agent by name
target_agent_name = "AIVirtualGuide"

# -----------------------------------------------------------
# 1. ELIGIBILITY
# -----------------------------------------------------------
eligible_tasks = []
for a, task in onto.isEligibleForTask.get_relations():
    if a.name == target_agent_name:
        eligible_tasks.append(task.name)

print("=== ELIGIBILITY ===")
if eligible_tasks:
    for t in eligible_tasks:
        print(f" - {t}")
else:
    print(" - None")

# -----------------------------------------------------------
# 2. ASSIGNMENT
# -----------------------------------------------------------
assigned_tasks = []
for a, task in onto.isAssignedToTask.get_relations():
    if a.name == target_agent_name:
        assigned_tasks.append(task.name)

print("\n=== ASSIGNMENTS ===")
if assigned_tasks:
    for t in assigned_tasks:
        print(f" - {t}")
else:
    print(" - None")

# -----------------------------------------------------------
# 3. CONTEXTUAL INFLUENCE (SWRL Rules 3a + 3b)
# -----------------------------------------------------------

# First, get the agent object by name
agent = None
for ind in onto.individuals():
    if ind.name == target_agent_name:
        agent = ind
        break

phenomena = []
if hasattr(agent, "hasPhenomenonConcept"):
    phenomena = [p.name for p in agent.hasPhenomenonConcept]

constraints = []
if hasattr(agent, "hasConstraintConcept"):
    constraints = [c.name for c in agent.hasConstraintConcept]

print("\n=== CONTEXTUAL INFLUENCE ===")

print("Phenomena:")
if phenomena:
    for p in phenomena:
        print(f"   * {p}")
else:
    print("   (none)")

print("Constraints:")
if constraints:
    for c in constraints:
        print(f"   * {c}")
else:
    print("   (none)")


=== ELIGIBILITY ===
 - TaskSemanticRetrieval
 - TaskRecommendArtifacts
 - TaskAnswerQuestions
 - TaskComputeSimilarity
 - TaskInterpretMultimodalInput
 - TaskExplainRecommendations
 - ExecRecommendArtifacts
 - ExecComputeSimilarity
 - ExecSemanticRetrieval
 - ExecAnswerQuestions
 - ExecExplainRecommendations
 - ExecInterpretInput

=== ASSIGNMENTS ===
 - ExecRecommendArtifacts
 - ExecComputeSimilarity
 - ExecSemanticRetrieval
 - ExecAnswerQuestions
 - ExecInterpretInput
 - ExecExplainRecommendations

=== CONTEXTUAL INFLUENCE ===
Phenomena:
   * KnowledgeMismatchPhenomenon
   * EngagementShiftPhenomenon
Constraints:
   * AccessibilityConstraint
   * PrivacyConstraint
   * TransparencyConstraint
